In [ ]:
import sqlite3
import csv

# Connect to the stock database
stock_conn = sqlite3.connect('stocks.db')
stock_cursor = stock_conn.cursor()

# Create stock table if it doesn't exist
stock_cursor.execute('''
    CREATE TABLE IF NOT EXISTS Stock(
        Ticker TEXT,
        Date TEXT,
        Open REAL,
        High REAL,
        Low REAL,
        Close REAL,
        AdjClose REAL,
        Volume INTEGER,
        CONSTRAINT unique_symbol_date UNIQUE (Ticker, Date)
    )
''')

# Read data from the CSV file
with open('stocks.csv') as f:
    reader = csv.reader(f)
    data = list(reader)

# Insert data into the table
for row in data:
    stock_cursor.execute("INSERT INTO Stock(Ticker, Date, Open, High, Low, Close, AdjClose, Volume) VALUES (?, ?, ?, ?, ?, ?, ?, ?)", row)

# Commit the changes
stock_conn.commit()

# Close the database connection
stock_conn.close()


In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('stocks.db')  
cursor = conn.cursor()

cursor.execute('SELECT * FROM Stock')

# Fetch all rows
rows = cursor.fetchall()

# Display the data
for row in rows:
    print(row)




In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('admins.db')
cursor = conn.cursor()

# Create tables if they don't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS admins (
        admin_id INTEGER PRIMARY KEY,
        username TEXT,
        password TEXT
    )
''')



# Function to create a new admin
def create_admin(username, password):
    cursor.execute('SELECT * FROM admins  WHERE username = ?', (username,))
    user = cursor.fetchone()
    if user:
        print("Username already exists. Please choose a different username.")
        return
    cursor.execute('INSERT INTO admins (username, password) VALUES (?, ?)', (username, password))
    conn.commit()
    print("User created successfully!")


# Main menu
def main_menu(logged_in_user):
    while True:
        print("\n==== Admin Management ====")
        if not logged_in_user:
            print("1. Register a new admin")
            print("2. Exit")
        choice = input("Enter your choice (1-2): ")

        #if not logged_in_user:
        if choice == '1':
            username = input("Enter a username: ")
            password = input("Enter a password: ")
            create_admin(username, password)
    
        elif choice == '2':
            break
        else:
            print("Invalid choice. Please try again.")

#Run the main menu with logged_in_user as None
main_menu(None)

# Close the database connection
conn.close()

In [ ]:
import sqlite3

# Connect to the stock database
stock_conn = sqlite3.connect('stocks.db')
stock_cursor = stock_conn.cursor()

# Create stock table if it doesn't exist
stock_cursor.execute ('''
    CREATE TABLE IF NOT EXISTS Stock (
        Ticker TEXT,
        Date TEXT,
        Open REAL,
        High REAL,
        Low REAL,
        Close REAL,
        AdjClose REAL,
        Volume INTEGER,
        CONSTRAINT unique_symbol_date UNIQUE (Ticker, Date)
    )
''')

# Connect to the users database
user_conn = sqlite3.connect('users.db')
user_cursor = user_conn.cursor()

# Create users table if it doesn't exist
user_cursor.execute('''
    CREATE TABLE IF NOT EXISTS Users (
        user_id INTEGER PRIMARY KEY,
        username TEXT,
        password TEXT
    )
''')

# Connect to the admins database
admin_conn = sqlite3.connect('admins.db')
admin_cursor = admin_conn.cursor()

# Create admins table if it doesn't exist
admin_cursor.execute('''
    CREATE TABLE IF NOT EXISTS admins (
        admin_id INTEGER PRIMARY KEY,
        username TEXT,
        password TEXT
    )
''')



# Function to check user credentials
def authenticate(username, password, user_type):
    if user_type == 'admin':
        cursor = admin_cursor
        table_name = 'admins'
    elif user_type == 'user':
        cursor = user_cursor
        table_name = 'Users'
    else:
        cursor = stock_cursor
        table_name = 'Stock'
    
    cursor.execute(f'SELECT * FROM {table_name} WHERE username = ? AND password = ?', (username, password))
    user = cursor.fetchone()
    
    if user:
        return True, user[0]  # Return True and user_id
    else:
        return False, None

# Function to create a new user
def create_user(username, password):
    user_cursor.execute('SELECT * FROM Users WHERE username = ?', (username,))
    user = user_cursor.fetchone()
    if user:
        print("Username already exists. Please choose a different username.")
        return
    user_cursor.execute('INSERT INTO Users (username, password) VALUES (?, ?)', (username, password))
    user_conn.commit()
    print("User created successfully!")

# CRUD Operations for Admin
def admin_operations(admin_id):
    while True:
        print("\n==== Admin CRUD Operations ====")
        print("1. Create Stock Entry")
        print("2. Read Stock Data")
        print("3. Update Stock Entry")
        print("4. Delete Stock Entry")
        print("5. Logout")

        admin_choice = input("Enter your choice (1-5): ")

        if admin_choice == '1':
            create_stock_entry(admin_id)
        elif admin_choice == '2':
            read_stock_data(admin_id)
        elif admin_choice == '3':
            update_stock_entry(admin_id)
        elif admin_choice == '4':
            delete_stock_entry(admin_id)
        elif admin_choice == '5':
            print("Logout successful!")
            break
        else:
            print("Invalid choice. Please try again.")

# Function to create a new stock entry
def create_stock_entry(user_id):
    ticker = input("Enter Ticker: ")
    date = input("Enter Date (YYYY-MM-DD): ")
    open_price = float(input("Enter Open Price: "))
    high = float(input("Enter High Price: "))
    low = float(input("Enter Low Price: "))
    close = float(input("Enter Close Price: "))
    adj_close = float(input("Enter Adj Close Price: "))
    volume = int(input("Enter Volume:"))

    stock_cursor.execute('''
        INSERT INTO Stock(Ticker, Date, Open, High, Low, Close, AdjClose, Volume)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (ticker, date, open_price, high, low, close, adj_close, volume))
    stock_conn.commit()
    print(f"Stock entry for {ticker} on {date} added successfully.")


# Function to read historical stock data based on criteria
def read_stock_data(user_id):
    ticker = input("Enter Ticker: ")
    end_date = input("Enter Date (YYYY-MM-DD): ")

    stock_cursor.execute('''
        SELECT * FROM Stock
        WHERE Ticker = ? AND Date = ?
    ''', (ticker, end_date))
    rows = stock_cursor.fetchall()

    if not rows:
        print(f"No data found for {ticker} on {end_date}.")
    else:
        for row in rows:
            print(row)
# Function to update an existing stock entry
def update_stock_entry(user_id):
    while True:
        ticker = input("Enter Ticker (or type 'exit' to stop): ")

        if ticker.lower() == 'exit':
            print("Exiting update mode.")
            break

        date = input("Enter Date (YYYY-MM-DD): ")

        # Display available columns for update
        print("Available columns for update:")
        print("1. Open")
        print("2. High")
        print("3. Low")
        print("4. Close")
        print("5. AdjClose")
        print("6. Volume")
        print("7. Exit")

        while True:
            column_choice = input("Enter the number corresponding to the column you want to update:(1 TO 7) ")

            if column_choice == '7':
                print("Exiting update mode.")
                break

            # Map the user's choice to the actual column name
            column_mapping = {
                '1': 'Open',
                '2': 'High',
                '3': 'Low',
                '4': 'Close',
                '5': 'AdjClose',
                '6': 'Volume'
            }

            selected_column = column_mapping.get(column_choice)
            if not selected_column:
                print("Invalid column choice. Please try again.")
                continue

            new_value = input(f"Enter the new value for {selected_column} (or type 'exit' to stop): ")

            if new_value.lower() == 'exit':
                print(f"Exiting update mode for {selected_column}.")
                break

            # Update the selected column for the specified stock entry
            stock_cursor.execute(f'''
                UPDATE Stock
                SET {selected_column} = ?
                WHERE Ticker = ? AND Date = ?
            ''', (new_value, ticker, date))

            stock_conn.commit()
            print(f"{selected_column} for {ticker} on {date} updated successfully.")



# Function to delete a stock entry
def delete_stock_entry(user_id):
    ticker = input("Enter Ticker: ")
    date = input("Enter Date (YYYY-MM-DD): ")

    stock_cursor.execute('''
        DELETE FROM Stock
        WHERE Ticker = ? AND Date = ?
    ''', (ticker, date))
    stock_conn.commit()
    print(f"Stock entry for {ticker} on {date} deleted successfully.")


# User Registration
def user_registration():
    print("\n==== User Registration ====")
    username = input("Enter a username: ")
    password = input("Enter a password: ")
    create_user(username, password)

# Main menu
def main_menu():
    while True:
        print("\n==== Main Menu ====")
        print("1. Admin Login")
        print("2. User Login")
        print("3. User Registration")
        print("4. Exit")

        choice = input("Enter your choice (1-4): ")

        if choice == '1':
            admin_username = input("Enter your admin username: ")
            admin_password = input("Enter your admin password: ")
            success, admin_id = authenticate(admin_username, admin_password, 'admin')
            if success:
                admin_operations(admin_id)
            else:
                print("Admin authentication failed. Please try again.")
        elif choice == '2':
            user_username = input("Enter your username: ")
            user_password = input("Enter your password: ")
            success, user_id = authenticate(user_username, user_password, 'user')
            if success:
                print("Get historical information of stocks")
                read_stock_data(user_id)
            else:
                print("User authentication failed. Please try again.")
        elif choice == '3':
            user_registration()
        elif choice == '4':
            print("Exiting the program. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

# Run the main menu
main_menu()

# Close the database connections
stock_conn.close()
user_conn.close()
